# Single Estimation

This notebook shows the full pipeline for estimating 6D pose of an object given 
RGB image $\mathcal{I}$ and matched depth map $\mathcal{D}$ and Object segmentation mask $\mathcal{M}$.

Major TODOS are as follows
* [ ] How we can pass a single reference image with its depth and segmentation mask and recover pose?
* [ ] Why .obj file was required and can we recover the same from a .ply point cloud?
* [ ]  

# V.V.I notes
* latenfusion.recon.models.Photographer: Takes in 3D latent space representation of an object and generates an image of that object
* latentfusion.recon.models.Sculptor: Takes 2D input data (RGB image, depth and segmentation masks) and
forms the volex-based 3D latent representation as described in Section 4.

To check how this pointcloud looks like, use Cloud compare
```/usr/bin/CloudCompare```

In [ ]:
# Imports
from matplotlib import pyplot as plt
import os
from pathlib import Path
# import numpy as np
import torch
# import torch.utils.model_zoo
import math

# LatentFusion
from latentfusion.recon.inference import Observation
from latentfusion.datasets.realsense import RealsenseDataset
import latentfusion.visualization as viz
from latentfusion.augment import gan_denormalize
from latentfusion import meshutils
from latentfusion import augment
from latentfusion.recon.inference import LatentFusionModel
from latentfusion.three.orientation import evenly_distributed_quats
import latentfusion.pose.estimation as pe

In [ ]:
# Setup global environment
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
MOPED_PATH = Path('datasets/moped')
num_ref_views = 8 # How many reference images to use?

In [ ]:
# Load pre-trained model
checkpoint = torch.load('weights/latentfusion-release.pth', weights_only=False)
model = LatentFusionModel.from_checkpoint(checkpoint, device)

In [ ]:
object_id = 'toy_plane' # Name of the object
frame_idx = 20

# Define path variables
# TODO check if we can get away without using the ground truth
input_scene_dir = MOPED_PATH / object_id / 'reference' # Ground-truth
target_scene_dir = MOPED_PATH / object_id / 'evaluation' # Input

# Why are we using object's point cloud?
pointcloud_path = input_scene_dir / 'integrated_registered_processed.obj'
obj = meshutils.Object3D(pointcloud_path)
pointcloud = torch.tensor(obj.vertices, dtype=torch.float32) # May not be used anywhere?
diameter = obj.bounding_diameter
object_scale = 1.0 / diameter
object_scale_to_meters = 1.0 / object_scale

print(f"diameter: {diameter}")
print(f"object_scale: {object_scale}")
print(f"object_scale_to_meters: {object_scale_to_meters}")

In [ ]:
# TODO find out why we need both .obj file and how it was made?
# TODO the function of RealsenseDataset class??
# Make listof paths
input_paths = [x for x in input_scene_dir.iterdir() if x.is_dir()]
#print(input_paths[:2]) # [PosixPath('datasets/moped/toy_plane/reference/06')]
input_dataset = RealsenseDataset(input_paths,
                                 image_scale=1.0,
                                 object_scale=object_scale,
                                 odometry_type='open3d')

In [ ]:
target_paths = sorted([x for x in target_scene_dir.iterdir() if x.is_dir()])
target_dataset = RealsenseDataset(target_paths,
                                  image_scale=1.0,
                                  object_scale=object_scale, # Can we just use 1.0 here??
                                  odometry_type='open3d',
                                  use_registration=True)
                                  

In [ ]:
# Batch input and target observations
# TODO what is Observation.from_dataset doing?
input_obs = Observation.from_dataset(input_dataset, inds=input_dataset.sample_evenly(num_ref_views))
target_obs = Observation.from_dataset(target_dataset, inds=list(range(len(target_dataset)))[frame_idx:frame_idx+1])

In [ ]:
# Preprocess observations
# TODO what does model.preprocess_observation do?
input_obs_pp = model.preprocess_observation(input_obs)
input_obs_pp_gt = model.preprocess_observation(input_obs)
target_obs_pp = model.preprocess_observation(target_obs)

## Step 1: Create latent representation of the new object.

In [ ]:
with torch.no_grad():
    z_obj = model.build_latent_object(input_obs_pp)

    # Visualize prediction.
    camera = input_obs_pp.camera.clone()
    y, z = model.render_latent_object(z_obj, camera.to(device))

# This is the reconstruction error. But for completely unseen objects, we will not be able to do this.
recon_error = (y['depth'].detach().cpu() - input_obs_pp_gt.depth).abs()
print('recon_error', recon_error.mean().item())

## Step 2: For given reference, find coarse estimation

In [ ]:
# Initialize a latentfusion.pose.estimation.CrossEntropyPoseEstimator model 
coarse_estimator = pe.load_from_config('configs/cross_entropy_latent.toml', model, return_camera_history=False, verbose=False)

# Given an object's 3D latent representation and input vector x = {I,D,M}, find camera poses that best describes the object?
# (Camera): One or multiple cameras representing the estimated pose.
coarse_camera = coarse_estimator.estimate(z_obj, target_obs[0]) 
camera_zoom = coarse_camera.zoom(None, model.camera_dist, model.input_size)

## Step 3: For the coarse estimates, obtain fine estimation